In [5]:
from chatbot.response_helper.get_sym import get_sym 

In [6]:
get_sym('KBANK')

'{\n    "type": "bubble",\n    "size": "giga",\n    "body": {\n        "type": "box",\n        "layout": "vertical",\n        "contents": [\n            {\n                "type": "text",\n                "text": "RESULT",\n                "weight": "bold",\n                "color": "#1DB446",\n                "size": "sm"\n            },\n            {\n                "type": "text",\n                "text": "Brown Store",\n                "weight": "bold",\n                "size": "xxl",\n                "margin": "md",\n                "contents": [\n                    {\n                        "type": "span",\n                        "text": "KBANK"\n                    }\n                ]\n            },\n            {\n                "type": "separator",\n                "margin": "xxl"\n            },\n            {\n                "type": "text",\n                "text": "BLOCK TRADE",\n                "weight": "bold",\n                "color": "#1DB446",\n              

In [7]:
import os
import sys

file_dir = os.getcwd()
chatbot_dir = os.path.join(file_dir, '..')
bloomberg_dir = os.path.join(chatbot_dir, '..')
dirs = [chatbot_dir, bloomberg_dir, file_dir]

sys.path.extend(d for d in dirs if d not in sys.path)

import polars as pl 
import pandas as pd 
import json 

data_dir = os.path.join(bloomberg_dir,'data')

from helpers.Flex import Flex
from helpers.Database import Database

import warnings 

warnings.filterwarnings('ignore')

    



In [8]:
sym = 'KBANK'

In [9]:
# --------------------------- Bloomberg Compopnent --------------------------- #
broker_list = pl.read_csv(os.path.join(data_dir, 'broker.csv')).to_pandas()
type_dict = broker_list[['broker', 'type']].set_index('broker').to_dict()['type']

bloomberg = pl.read_csv(os.path.join(data_dir, 'bloomberg.csv')).to_pandas()
bloomberg = bloomberg[bloomberg['symbol'] == sym]
bloomberg['date'] = pd.to_datetime(bloomberg['date'])
bloomberg = bloomberg[bloomberg['date'].dt.year == pd.Timestamp.now().year]
bloomberg['date'] = bloomberg['date'].dt.strftime('%Y-%m-%d')

bloomberg['target'] = pd.to_numeric(bloomberg['target'], errors='coerce')

if bloomberg.empty:
    bloomberg_foreign_component = None
    bloomberg_local_component = None
else:  
    bloomberg_df = bloomberg[['broker', 'comment', 'target', 'date']]
    bloomberg_df['type'] = bloomberg_df['broker'].map(type_dict)
    
    bloomberg_df['date'] = pd.to_datetime(bloomberg_df['date'])
    bloomberg_df.sort_values('date', ascending=False, inplace=True)
    bloomberg_df['date'] = bloomberg_df['date'].dt.strftime('%Y-%m-%d')
    bloomberg_df.drop_duplicates(subset=['broker', 'comment', 'target'], keep='first', inplace=True)

    for col in bloomberg_df.columns:
        if col == 'target':
            bloomberg_df[col] = bloomberg_df[col].apply(lambda x: f'{x:,.2f}')
        else:
            bloomberg_df[col] = bloomberg_df[col].apply(lambda x: f'{x}')

    bloomberg_df_local = bloomberg_df[bloomberg_df['type'] == 'local'].drop(columns='type').reset_index(drop=True)
    bloomberg_df_foreign = bloomberg_df[bloomberg_df['type'] == 'foreign'].drop(columns='type').reset_index(drop=True)

In [10]:
# ------------------------------- IM Component ------------------------------- #
im = pl.read_csv(os.path.join(data_dir, 'im.csv')).to_pandas()
im_df = im[im['symbol'] == sym]
if im_df.empty:
    im_component = None
else:
    im_df.drop(columns=['symbol'], inplace=True)
    im_df.columns = ['ขั้นต่ำ', 'Leverage', 'ใช้เงิน']

    im_df['ขั้นต่ำ'] = im_df['ขั้นต่ำ'].apply(lambda x: f'{x:,.0f}')
    im_df['ใช้เงิน'] = im_df['ใช้เงิน'].apply(lambda x: f'{x:,.0f}')
    im_df['Leverage'] = im_df['Leverage'].apply(lambda x: f'{x}')
    im_df = im_df.T.reset_index()
    im_df = im_df.T.reset_index(drop=True).T
    
    im_df[2] = ['สัญญา', 'เท่า', 'บาท']
    

# ------------------------------ Summary Component ---------------------------- #
avg_sep_local = pd.DataFrame(data=[ bloomberg_df_local['target'].astype(float).mean().round(2)], columns=["Average Target Price"])
avg_sep_foreign = pd.DataFrame(data=[ bloomberg_df_foreign['target'].astype(float).mean().round(2)], columns=["Average Target Price"])

most_common_local = bloomberg_df_local.comment.value_counts().iloc[[0]].reset_index()
most_common_foreign = bloomberg_df_foreign.comment.value_counts().iloc[[0]].reset_index()


most_common_local['Most Common Comment'] = most_common_local['comment'].astype(str) + ' (' + most_common_local['count'].astype(str) + ')'
most_common_foreign['Most Common Comment'] = most_common_foreign['comment'].astype(str) + ' (' + most_common_foreign['count'].astype(str) + ')'
most_common_local = most_common_local[['Most Common Comment']].T.reset_index().T.reset_index(drop=True).T
most_common_foreign = most_common_foreign[['Most Common Comment']].T.reset_index().T.reset_index(drop=True).T